In [1]:
import numpy as np
import pandas as pd

In [ ]:
# try : score
#1 ## lr = 1e-3 : 0.66- 
#2 ## lr = 1e-4 : 0.68-
#3 ## 2 + remove SkinThickness column : 7.09-
#4 ## 3 + DiabetesPedigreeFunction * 100 : 6.88-
#5 ## 4 + Pregnancies * 10 : 0.722-
#6 ## 5 + reduce epochs to 5000 : 0.722-
#7 ## 6 + remove Glucose column : 0.67-
#8 ## 6 + lr = 1e-2 : 0.53-
#9 ## 6 + increase epochs to 100000 : 0.72-
#10## all col + increase epochs to 100000 : 0.71-
#11## original col + increase epochs to 1000000 : 


In [2]:
import torch
import torch.optim as optim
# For reproducibility
torch.manual_seed(4)

In [4]:
train_df = pd.read_csv("./당뇨병예측하기_data/train.csv")
test_df = pd.read_csv("./당뇨병예측하기_data/test.csv")
submit_df = pd.read_csv("./당뇨병예측하기_data/submit.csv")
print(train_df.head())
print(test_df.head())
print(submit_df.head())

   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            1       95             60             18       58  23.9   
1            5      105             72             29      325  36.9   
2            0      135             68             42      250  42.3   
3            4      131             68             21      166  33.1   
4            1      103             30             38       83  43.3   

   DiabetesPedigreeFunction  Age  Diabetes  
0                     0.260   22         0  
1                     0.159   28         0  
2                     0.365   24         1  
3                     0.160   28         0  
4                     0.183   33         0  
   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6       98             58             33      190  34.0   
1            2      112             75             32        0  35.7   
2            2      108             64              0        0  30.8   
3        

In [25]:
x_train = np.array([train_df['Pregnancies'],train_df['Glucose'],train_df['BloodPressure'],train_df['SkinThickness'],train_df['Insulin'],train_df['BMI'],train_df['DiabetesPedigreeFunction'],train_df['Age']])
# x_train = np.array([train_df['Pregnancies']*10,train_df['Glucose'],train_df['BloodPressure'],train_df['Insulin'],train_df['BMI'],train_df['DiabetesPedigreeFunction']*100,train_df['Age']])
x_train = x_train.T
x_train = torch.FloatTensor(x_train)
y_train = torch.FloatTensor(train_df['Diabetes'])
print(x_train.shape)
print(y_train.shape)

torch.Size([537, 8])
torch.Size([537, 1])


In [49]:
# 모델 초기화
W = torch.zeros((8, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)


# Loss Function
loss = torch.nn.BCELoss()
# optimizer 설정
# optimizer = optim.SGD([W, b], lr=1e-3) 
optimizer = optim.SGD([W, b], lr=1e-4) 


nb_epochs = 1000000
for epoch in range(nb_epochs + 1):

    # Cost 계산
    ## use torch.sigmoid instead torch.nn.functional.sigmoid 
    hypothesis = torch.sigmoid(x_train.matmul(W) + b) # or .mm or @
    cost = loss(hypothesis,y_train.unsqueeze(1))
#     cost = -(y_train * torch.log(hypothesis) + 
#              (1 - y_train) * torch.log(1 - hypothesis)).mean()

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    ## cost를 개선한다는 의미
    optimizer.step()
    ## W를 갱신 

    # 100번마다 로그 출력
    if epoch % 10000 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))


Epoch    0/10000 Cost: 0.693147
Epoch 1000/10000 Cost: 0.622137
Epoch 2000/10000 Cost: 0.620818
Epoch 3000/10000 Cost: 0.619854
Epoch 4000/10000 Cost: 0.619112
Epoch 5000/10000 Cost: 0.618530
Epoch 6000/10000 Cost: 0.618065
Epoch 7000/10000 Cost: 0.617684
Epoch 8000/10000 Cost: 0.617364
Epoch 9000/10000 Cost: 0.617089
Epoch 10000/10000 Cost: 0.616846


In [50]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b)
prediction = hypothesis >= torch.FloatTensor([0.5])
correct_prediction = prediction.int() == y_train.unsqueeze(1)
accuracy = correct_prediction.sum().item() / len(correct_prediction)
print('The model has an accuracy of {:2.2f}% for the training set.'.format(accuracy * 100))

The model has an accuracy of 68.16% for the training set.


In [33]:
x_test = np.array([test_df['Pregnancies'],test_df['Glucose'],test_df['BloodPressure'],test_df['SkinThickness'],test_df['Insulin'],test_df['BMI'],test_df['DiabetesPedigreeFunction'],test_df['Age']])
# x_test = np.array([test_df['Pregnancies']*10,test_df['Glucose'],test_df['BloodPressure'],test_df['Insulin'],test_df['BMI'],test_df['DiabetesPedigreeFunction'] * 100,test_df['Age']])
x_test = x_test.T
x_test = torch.FloatTensor(x_test)

In [39]:
hypothesis = torch.sigmoid(x_test.matmul(W) + b)
prediction = hypothesis >= torch.FloatTensor([0.5])
prediction = prediction.int()
submit_df['Diabetes'] = prediction.detach().numpy()
submit_df.head()

,ID,Diabetes
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [40]:
submit_df.to_csv("./당뇨병예측하기_data/submit.csv",index=False)